In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval

In [2]:
pubs_author_funder = pd.read_csv("..//..//..//data//Data//DerivedData//CleanedRawData//pubs_author_funder.csv")
pubs_author_funder['author_country'] = pubs_author_funder['author_country'].apply(literal_eval)
pubs_author_funder['funder_country'] = pubs_author_funder['funder_country'].apply(
    lambda x: literal_eval(x) if pd.notnull(x) else x)

/home/miaoli/conda/envs/funding/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
pubs_author_funder.head()

,id,author_country,funder_country
0,30507292,[Turkey],[Turkey]
1,30574690,"[United States, France]","[United States, United States, United States, ..."
2,30574691,[Russia],[Russia]
3,30585095,"[Spain, Spain]","[Spain, Spain, Spain, Spain, Spain, Spain]"
4,30585097,"[Germany, Spain]",[Spain]


In [4]:
pubs_author_funder = pubs_author_funder.fillna(0)
pubs_author_funder['num_funder'] = pubs_author_funder['funder_country'].apply(
    lambda x:len(set(x)) if x!=0 else 0)

In [5]:
pubs_author_funder['funding_unit']=pubs_author_funder['funder_country'].apply(
lambda x:len(x) if x!=0 else 0)

In [6]:
pubs_author_funder.head()

,id,author_country,funder_country,num_funder,funding_unit
0,30507292,[Turkey],[Turkey],1,1
1,30574690,"[United States, France]","[United States, United States, United States, ...",1,4
2,30574691,[Russia],[Russia],1,1
3,30585095,"[Spain, Spain]","[Spain, Spain, Spain, Spain, Spain, Spain]",1,6
4,30585097,"[Germany, Spain]",[Spain],1,1


In [8]:
funder_df=pubs_author_funder[
    pubs_author_funder.num_funder>0][['id','funder_country','num_funder']]
funder_df.head()

,id,funder_country,num_funder
0,30507292,[Turkey],1
1,30574690,"[United States, United States, United States, ...",1
2,30574691,[Russia],1
3,30585095,"[Spain, Spain, Spain, Spain, Spain, Spain]",1
4,30585097,[Spain],1


In [9]:
funder_df = funder_df.explode('funder_country')
funder_df.head()

,id,funder_country,num_funder
0,30507292,Turkey,1
1,30574690,United States,1
1,30574690,United States,1
1,30574690,United States,1
1,30574690,United States,1


In [10]:
funder_df['cofund'] = funder_df['num_funder'].apply(lambda x:1 if x>1 else 0)
funder_df

,id,funder_country,num_funder,cofund
0,30507292,Turkey,1,0
1,30574690,United States,1,0
1,30574690,United States,1,0
1,30574690,United States,1,0
1,30574690,United States,1,0
...,...,...,...,...
7739778,65301284,Canada,2,1
7739778,65301284,Turkey,2,1
7739779,65301319,China,1,0
7739780,65301320,United Kingdom,1,0


In [13]:
cofund_df = funder_df.groupby(['id','funder_country','cofund']).size().reset_index()
cofund_df.head()

,id,funder_country,cofund,0
0,30507292,Turkey,0,1
1,30574690,United States,0,4
2,30574691,Russia,0,1
3,30585095,Spain,0,6
4,30585097,Spain,0,1


In [14]:
cofund_df.columns=['id','funder_country','cofund','unit']
cofund_df.head()

,id,funder_country,cofund,unit
0,30507292,Turkey,0,1
1,30574690,United States,0,4
2,30574691,Russia,0,1
3,30585095,Spain,0,6
4,30585097,Spain,0,1


In [15]:
unitavg_df = cofund_df.groupby(
    ['funder_country','cofund'])['unit'].mean().reset_index()
unitavg_df.head()

,funder_country,cofund,unit
0,Afghanistan,0,1.000000
1,Afghanistan,1,1.000000
2,Albania,0,1.000000
3,Albania,1,1.000000
4,Algeria,0,1.180317


In [17]:
unitavg_df = unitavg_df.pivot(index="funder_country",columns='cofund',values='unit')
unitavg_df.columns=['selffund','cofund']
unitavg_df.head()

,selffund,cofund
funder_country,,
Afghanistan,1.000000,1.000000
Albania,1.000000,1.000000
Algeria,1.180317,1.101620
Andorra,1.111111,1.064516
Angola,1.000000,1.568182


In [19]:
unitavg_df.sort_values(by="cofund", ascending=False).head(30)

,selffund,cofund
funder_country,,
China,3.312176,2.851380
United States,2.487977,2.791090
Finland,2.565194,2.700997
Sweden,2.546054,2.500422
Spain,2.401591,2.463792
Brazil,2.487162,2.250345
Portugal,2.202933,2.245178
Argentina,2.443038,2.187019
Canada,2.141663,2.105634
